In [5]:
import openai
import os
import re
import pandas as pd
import spacy
from ipywidgets import FloatProgress
from tqdm import tqdm

In [6]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [11]:
os.environ["OPENAI_API_KEY"] = "XXXXX"  # replace with yours

In [12]:
movies = pd.read_csv("netflix_titles.csv")
movies = movies.sample(n=1000)

In [14]:
def predict_genres(movie_description):
    prompt = f"Predict the top three genres (only genres, not descriptions) for a movie with the following description: {movie_description}"
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",  # You can use the GPT-3 model for this task
      prompt=prompt,
      max_tokens=50,
      n=1,
      stop=None,
      temperature=0.2
    )
    predicted_genres = response.choices[0].text.strip()
    return predicted_genres

In [15]:
# Create an empty list to store the predicted genres
all_predicted_genres = []

# Create an empty set to store unique genres
unique_genres_set = set()

# Iterate through the movie descriptions
for index, row in tqdm(movies.iterrows(), total=movies.shape[0]):
    
        # Get the movie description
        movie_description = row['description']
    
        # Predict the genres for the movie description
        predicted_genres = predict_genres(movie_description)
    
        # Extract genres from the text
        predicted_genres_tokens = nlp(predicted_genres)
        predicted_genres_tokens = predicted_genres_tokens.text
        
        # Use regular expression to extract genres
        genres_with_numbers = re.findall(r'\d+\.\s*([^\n]+)', predicted_genres_tokens)
        
        # Remove leading/trailing whitespaces from each genre
        predicted_genres = [genre.strip().lower() for genre in genres_with_numbers]
    
        # Update the set of unique genres
        unique_genres_set.update(predicted_genres)

# Convert the set of unique genres back to a list
all_unique_genres = list(unique_genres_set)

100%|█████████████████████████████████████████| 100/100 [00:46<00:00,  2.17it/s]


In [16]:
all_unique_genres = pd.DataFrame(all_unique_genres,columns=['genre'])

In [17]:
all_unique_genres.head()

,genre
0,social commentary
1,children's music
2,fantasy
3,animation
4,family


In [31]:
all_unique_genres.to_csv("genres_taxonomy_quick.csv")